In [6]:
from __future__ import annotations

import json
from pathlib import Path
from typing import Any, Dict, Iterable, List

import pandas as pd

from mainnet_launch.constants import WORKING_DATA_DIR, AutopoolConstants
from mainnet_launch.database.postgres_operations import (
    get_full_table_as_df_with_block,
    get_full_table_as_df,
    get_full_table_as_df_with_tx_hash,
)
from mainnet_launch.database.schema.full import (
    DestinationStates,
    Destinations,
    AutopoolDestinations,
    RebalanceEvents,
    RebalancePlans,
    Blocks,
)


from mainnet_launch.constants import AUTO_ETH, BASE_ETH


def _flatten_json(obj: Any, prefix: str = "", sep: str = ".") -> Dict[str, Any]:
    """
    Recursively flatten dict/list JSON into a single-level dict.

    - dict keys become: prefix.key
    - list items become: prefix[0], prefix[1], ...
    """
    out: Dict[str, Any] = {}

    if isinstance(obj, dict):
        for k, v in obj.items():
            key = f"{prefix}{sep}{k}" if prefix else str(k)
            out.update(_flatten_json(v, key, sep=sep))
        return out

    if isinstance(obj, list):
        for i, v in enumerate(obj):
            key = f"{prefix}[{i}]"
            out.update(_flatten_json(v, key, sep=sep))
        return out

    # primitives (str/int/float/bool/None)
    out[prefix] = obj
    return out


def load_augmented_plans_as_df(
    autopool: AutopoolConstants,
    *,
    save_root: Path = WORKING_DATA_DIR,
    glob_pattern: str = "*.json",
    flatten_sep: str = ".",
) -> pd.DataFrame:
    """
    Reads all augmented plan JSONs for an autopool from:
      WORKING_DATA_DIR / f"{autopool.name}_augmented_plans"

    Returns a DataFrame with:
      - one row per plan
      - ALL values, flattened (including destinationIn/Out, destinationOutSummaryStats, start_vp/end_vp, etc.)
      - a couple convenience columns for file path + json key
    """
    plans_dir = save_root / f"{autopool.name}_augmented_plans"
    if not plans_dir.exists():
        raise FileNotFoundError(f"Augmented plans dir not found: {plans_dir}")

    rows: List[Dict[str, Any]] = []
    json_paths = sorted(plans_dir.glob(glob_pattern))

    for p in json_paths:
        with open(p, "r") as f:
            plan = json.load(f)

        for k in ["sod", "steps", "addRank", "rebalanceTest"]:
            plan.pop(k, None)
        flat = _flatten_json(plan, sep=flatten_sep)

        # a couple helpful traceability fields
        flat["_file_path"] = str(p)
        flat["_file_name"] = p.name

        # often present in your saved json already, but keep safe
        if "rebalance_plan_json_key" not in flat:
            flat["rebalance_plan_json_key"] = p.stem

        rows.append(flat)

    df = pd.DataFrame(rows)

    df["predicted_fee_and_base_out"] = (
        df["destinationOutSummaryStats.out.feeApr"] + df["destinationOutSummaryStats.out.baseApr"]
    ) * 100
    df["predicted_fee_and_base_in"] = (
        df["destinationOutSummaryStats.in.feeApr"] + df["destinationOutSummaryStats.in.baseApr"]
    ) * 100

    df["actual_fee_and_base_out"] = df["out_fee_and_base"]
    df["actual_fee_and_base_in"] = df["in_fee_and_base"]

    destinations_to_name = {
        d.destination_vault_address: d.name for d in get_full_table_as_df(Destinations).itertuples()
    }
    df["out_name"] = df["destinationOut"].map(destinations_to_name)
    df["in_name"] = df["destinationIn"].map(destinations_to_name)

    df.to_csv(f"{autopool.name}_augmented_plans.csv", index=False)
    return df


# Example usage:
df = load_augmented_plans_as_df(AUTO_ETH)
df_base_eth = load_augmented_plans_as_df(BASE_ETH)
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_white"

In [2]:
import numpy as np
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_white"


def add_diagonal_line(fig, start=0, end=12):
    """Add a diagonal x=y reference line to a plotly figure"""
    fig.add_shape(
        type="line",
        x0=start,
        y0=start,
        x1=end,
        y1=end,
        line=dict(dash="dash", color="gray", width=2),
    )
    return fig


def add_estimation_labels(fig, start=0, end=12, band=1.0):
    """Add 'Overestimated'/'Underestimated' labels + +/- band lines (y=x±band)."""
    fig.add_annotation(
        x=0.95,
        y=0.05,
        xref="paper",
        yref="paper",
        text="Overestimated",
        showarrow=False,
        font=dict(size=14, color="black"),
        align="right",
    )
    fig.add_annotation(
        x=0.05,
        y=0.95,
        xref="paper",
        yref="paper",
        text="Underestimated",
        showarrow=False,
        font=dict(size=14, color="black"),
        align="left",
    )

    # y = x + band
    fig.add_shape(
        type="line",
        x0=start,
        y0=start + band,
        x1=end,
        y1=end + band,
        line=dict(dash="dot", color="black", width=1),
    )
    # y = x - band
    fig.add_shape(
        type="line",
        x0=start + band,
        y0=start,
        x1=end + band,
        y1=end,
        line=dict(dash="dot", color="black", width=1),
    )
    return fig


def add_helpers(fig, start=0, end=12, band=1.0):
    fig = add_diagonal_line(fig, start=start, end=end)
    fig = add_estimation_labels(fig, start=start, end=end, band=band)
    return fig


def _scatter_pred_vs_actual(
    df,
    *,
    pred_col: str,
    actual_col: str,
    title: str,
    hover_cols: list[str],
    label_col: str | None = None,
    exclude_negative_actual: bool = True,
):
    d = df.copy()

    # keep only finite points
    d = d[np.isfinite(d[pred_col]) & np.isfinite(d[actual_col])]

    # exclude outliers where actual is negative
    if exclude_negative_actual:
        d = d[d[actual_col] >= 0]

    if len(d) == 0:
        print(f"{title}: no data after filters")
        return None

    # choose axis range based on filtered data (robust-ish)
    lo = float(min(d[pred_col].min(), d[actual_col].min()))
    hi = float(max(d[pred_col].max(), d[actual_col].max()))

    # if APR is percent-like and you want to clamp to something sane, do it here
    # (otherwise it'll just use your data range)
    pad = 0.05 * (hi - lo) if hi > lo else 1.0
    start = max(0.0, lo - pad)  # keep at >=0 since we're excluding negative actual
    end = hi + pad

    fig = px.scatter(
        d,
        x=pred_col,
        y=actual_col,
        color=label_col if label_col and (label_col in d.columns) else None,
        hover_data=hover_cols,
        title=title,
    )
    fig.update_xaxes(range=[start, end])
    fig.update_yaxes(range=[start, end])
    fig = add_helpers(fig, start=start, end=end, band=1.0)
    fig.show()
    return fig


# --- Example usage (AUTO_ETH) ---
# df = load_augmented_plans_as_df(AUTO_ETH)

_scatter_pred_vs_actual(
    df,
    pred_col="predicted_fee_and_base_out",
    actual_col="actual_fee_and_base_out",
    title="AUTO_ETH Out: Predicted vs Actual Fee+Base (30d)",
    hover_cols=[],
    label_col="out_name",
    exclude_negative_actual=True,
)

_scatter_pred_vs_actual(
    df,
    pred_col="predicted_fee_and_base_in",
    actual_col="actual_fee_and_base_in",
    title="AUTO_ETH In: Predicted vs Actual Fee+Base (30d)",
    hover_cols=[],
    label_col="in_name",
    exclude_negative_actual=True,
)

In [5]:
# todo verify each of these virtual price functions for the outliers. see it they are right

In [ ]:
#

In [3]:
_scatter_pred_vs_actual(
    df_base_eth,
    pred_col="predicted_fee_and_base_out",
    actual_col="actual_fee_and_base_out",
    title="AUTO_ETH Out: Predicted vs Actual Fee+Base (30d)",
    hover_cols=[],
    label_col="out_name",
    exclude_negative_actual=True,
)

_scatter_pred_vs_actual(
    df_base_eth,
    pred_col="predicted_fee_and_base_in",
    actual_col="actual_fee_and_base_in",
    title="AUTO_ETH In: Predicted vs Actual Fee+Base (30d)",
    hover_cols=[],
    label_col="in_name",
    exclude_negative_actual=True,
)

In [4]:
df_auto_eth

NameError: name 'df_auto_eth' is not defined